## Libraries


In [1]:
import csv
import json
import pandas as pd

## Data

In [2]:
# Loading the JSON data of food details
    
with open('food.json', "r") as file:    
    food_items = json.load(file)

# List of food items

foods = list(food_items.keys())

# MET value is used to estimate calories burned for a particular physical activity

met_value = {  
    "No activity": 0,
    "Walking": 3.6,
    "Running": 10.0,
    "Cycling": 8.0,
    "Swimming": 6.0,
    "Aerobics": 7.5,
    "Strength training": 5.5,
    "HIIT": 8.0,
    "Football": 10.0,
}

## User-defined functions

In [3]:
def create_account():
    
    # USERNAME & NAME
        
    username = input('Enter username: ')                
            
    name = input("Enter name: ")
        
    # AGE

    while True:
            try:
                age = int(input('Enter age: '))
                if 5 <= age <= 120:
                    break
                else:
                    print('Invalid age. Please enter a value between 5 and 120.')
            except ValueError:
                print('Invalid input. Please enter a valid age as a number.')
                
                
        # WEIGHT

        
    while True:
            try:
                weight = float(input('Enter weight in kg: '))
                if 10 <= weight <= 600:
                    break
                else:
                    print('Invalid weight. Please enter a value between 10 and 600.')
            except ValueError:
                print('Invalid input. Please enter a valid weight as a number.')
             
        
        # HEIGHT
        
    while True:
            try:
                height = int(input('Enter height in cm: '))
                if height >= 0:
                    break
                else:
                    print('Invalid height. Please enter a non-negative value.')
            except ValueError:
                print('Invalid input. Please enter a valid height as a number.')
                
        # GENDER
    valid_options = ["male", "female"]
    while True:
            gender = input('Enter gender: ')
            if gender.lower() in valid_options:
                break
            else:
                print('Invalid gender. Please enter only male or female.')
        
    
        # ACTIVITY LEVEL
        
    valid_options = ["sedentary", "lightly active", "moderately active", "very active"]
    while True:
            activity_level = input('Enter activity level (sedentary, lightly active, moderately active, very active): ')
            if activity_level.lower() in valid_options:
                break
            else:
                print('Invalid activity level. Please choose from the given options.')
        
       
        #  GOAL WEIGHT
        
    while True:
            try:
                goal_weight = float(input('Enter goal weight in kg: '))
                if 10 <= goal_weight <= 600:
                    break
                else:
                    print('Invalid goal weight. Please enter a value between 10 and 600.')
            except ValueError:
                print('Invalid input. Please enter a valid weight as a number.')
        
        # Creating User Object
        
    user = UserProfile(username, name, age, gender, weight, height, activity_level, goal_weight)
    print('New account successfully created')
                
    return username, name, age, gender, weight, height, activity_level, goal_weight
        
    

# Gives interpretation of BMI value

def bmi_interpretation(bmi):
        if bmi < 18.5:
            interpretation = "Underweight"
        elif 18.5 <= bmi < 25:
            interpretation = "Normal weight"
        elif 25 <= bmi < 30:
            interpretation = "Overweight"
        else:
            interpretation = "Obese"

        return interpretation
    
# Estimates calories burned using MET values 
                        
def calculate_calories_burned(met_value, weight, duration):
    calories_burned = met_value * float(weight) * duration
    return calories_burned


# Calculates calories and macronutrients consumed 

def meal_calorie_calculator(items):
    
    total_calories = 0
    total_carbs = 0
    total_fats = 0
    total_proteins = 0

    for item in range(items):
        
        food = input('Enter food item: ')
        
        amount = float(input('Enter amount of food consumed in grams: '))
        

        food_calorie = food_items[food]['calories']
        total_calories += food_calorie*amount

        food_carbs = food_items[food]['carbohydrates']
        total_carbs += food_carbs*amount

        food_fats = food_items[food]['fats']
        total_fats += food_fats*amount

        food_proteins = food_items[food]['proteins']
        total_proteins += food_proteins*amount

    return total_calories, total_fats, total_carbs, total_proteins


def calculate_recommended_macronutrient_intake(total_calories):
    
    carb_calories = total_calories * 0.5
    protein_calories = total_calories * 0.2
    fat_calories = total_calories * 0.3

    carb_grams = carb_calories / 4
    protein_grams = protein_calories / 4
    fat_grams = fat_calories / 9

    return carb_grams, protein_grams, fat_grams

## User Profile Class

In [4]:
class UserProfile:

    def __init__(self,user_name, name, age, gender, weight, height, activity_level, goal_weight):
        
        self.user_name = user_name
        self.name = name
        self.age = age
        self.gender = gender
        self.weight = weight  # in kg
        self.height = height  # in cm
        self.activity_level = activity_level  # sedentary, lightly active, moderately active, very active
        self.goal_weight = goal_weight
        self.bmi = round(self.weight / ((self.height / 100) ** 2), 1)  # BMI formula
        self.add_to_csv()

    
    # Stores user profile data in CSV
    
    def add_to_csv(self):
        
        with open('user_data.csv', 'a', newline='') as file:          
            writer = csv.writer(file)
            writer.writerow([self.user_name, self.name, self.age, self.weight, self.height, self.gender, self.activity_level,
                             self.goal_weight, self.bmi, self.daily_calories(), self.recommended_calorie_consumption()])  # Writing data rows
    
            
    # Calculates daily calorie required to consume to maintain current weight
    
    def daily_calories(self):  
        
        #  Mifflin-St. Jeor equation to calculate calorie intake
        
        if self.gender == "male":
            bmr = 10 * self.weight + 6.25 * self.height - 5 * self.age + 5
            
        elif self.gender == "female":
            bmr = 10 * self.weight + 6.25 * self.height - 5 * self.age - 161
            

        # Accounting for physical activity levels

        if self.activity_level == "sedentary":
            calorie_req = bmr * 1.2
            
        elif self.activity_level == "lightly active":
            calorie_req = bmr * 1.375
            
        elif self.activity_level == "moderately active":
            calorie_req = bmr * 1.55
            
        elif self.activity_level == "very active":
            calorie_req = bmr * 1.725

        return round(calorie_req)

   
    # Calories to consume to reach goal weight 
    
    def recommended_calorie_consumption(self): 
        
        calories = self.daily_calories()
        
        if self.weight>self.goal_weight:
            calorie_intake = calories - 500
            
        elif self.weight<self.goal_weight:
            calorie_intake = calories + 500
            
        else:
            calorie_intake = calories   
            
        return calorie_intake


## Interface

In [6]:
def main():
    
    print('Welcome to the calorie tracker app.')
      
    while True:

        # DISPLAY MENU OPTIONS
        
        print('')
        print("FUNCTIONS")
        print("0. Login/ Sign Up")
        print("1. Display User Information")
        print("2. Enter food details")
        print("3. Display calorie consumption information for the day")
        print("4. Display past daily calorie consumption data")
        print("5 EXIT")
        print("")

        ch = int(input('Enter choice: '))
        
        if ch == 0:
           
    
            # SIGN UP

            acc = input('Do you want to create a new account?(Y/N): ').upper()

            if acc=='Y':
                print('Creating your account')  
                username, name, age, gender, weight, height, activity_level, goal_weight = create_account()



            # LOGGING IN

            print('Login to your account')


            username = input('Enter username: ')

        if ch==1:
            with open('user_data.csv', 'r', ) as file:
                    data = csv.reader(file)
                    for row in data:
                        if row[0] == username:
                            print("")
                            print('USER INFORMATION')
                            print("Username:",row[0])
                            print("Name:", row[1])
                            print('Age:', row[2])
                            print('Weight:',row[3])
                            print('Height:',row[4])
                            print('Gender:', row[5])
                            print('Activity level:', row[6])
                            print('Goal weight:', row[7])
                            print('BMI:',row[8], ', Interpretation:', bmi_interpretation((float(row[8]))))
                            print('Daily calorie intake:',row[9])
                            print('Recommended calorie intake:', row[10])


        elif ch ==2:
            
            print('Food Consumed Today')

            date = input('Enter date in format dd-mm-yy: ')

            items = int(input('Enter number of food items consumed today: '))
            
            print('Available food items: ')
            print(foods)

            total_calories, total_fats, total_carbs, total_proteins = meal_calorie_calculator(items)

            # Saving data to file
            
            with open('data.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([username,date,total_calories,total_carbs, total_proteins,total_fats])

            print('Food data has been entered')
            print("")

            
        elif ch==3:
            
            # ACTIVITY 
            
            valid_options = ["No activity", "Walking","Running","Cycling","Swimming","Aerobics","Strength training","HIIT","Football"]
            while True:
                activity = input('Enter activity from ["No activity", "Walking","Running","Cycling","Swimming","Aerobics","Strength training","HIIT","Football"]:')
                if activity in valid_options:
                    break
                else:
                    print('Invalid activity. Please choose from the given options.')
                    
            # DURATION
            
            duration = float(input('Enter duration of activity performed in hours: '))

            carb_grams, protein_grams, fat_grams = calculate_recommended_macronutrient_intake(total_calories)
            
            print('Total carbohydrates consumed:', total_carbs,'  Recommended carbohydrates consumption:',carb_grams)
            print('Total proteins consumed:', total_proteins, '  Recommended proteins consumption:',protein_grams)
            print('Total fats consumed:', total_fats,'  Recommended fat consumption:',fat_grams)
            print('Total calories consumed:', total_calories)
            
            with open('user_data.csv', 'r', ) as file:
                    data = csv.reader(file)
                    for row in data:
                        if row[0] == username: 
                            weight = row[3]
                            recommended_calories = float(row[10])
                            
            # Calories burned by performing activity

            met = met_value[activity]
            calories_burned = calculate_calories_burned(met, weight, duration)
            print('Total calories burned:',calories_burned)
            
            # Net calories consumed and recommended calories to be consumed

            net_calories = total_calories-calories_burned 
            print('Net calorie consumption of the day:{:.2f}'.format(net_calories))
            print('Recommended calorie consumption: {:.2f}'.format(recommended_calories))
            
            # Checking whether calories consumed in more or less than recommended

            if net_calories<recommended_calories:
                print(f'You have consumed {recommended_calories-net_calories} less calories than recommeded for achieving your goal weight.')
            elif net_calories> recommended_calories:
                print(f'You have consumed {net_calories-recommended_calories} more calories than recommended for achieving your goal weight.')

            else:
                print('You have consumed the right amount of calories for achieving your goal weight. Well done!')

        elif ch==4:
            
            print("")
            print('Past calorie consumtion data')
            
            # Using pandas dataframe to display past user data
        
            df = pd.read_csv('data.csv') 
            print(df[df.Username==username])

        elif ch==5:
            
            print('EXIT')
            break
            
        else:
            print('Invalid choice')



main()






Welcome to the calorie tracker app.

FUNCTIONS
0. Login/ Sign Up
1. Display User Information
2. Enter food details
3. Display calorie consumption information for the day
4. Display past daily calorie consumption data
5 EXIT

Enter choice: 0
Do you want to create a new account?(Y/N): Y
Creating your account
Enter username: ado19
Enter name: Adrian
Enter age: 245
Invalid age. Please enter a value between 5 and 120.
Enter age: 25
Enter weight in kg: 90
Enter height in cm: 190
Enter gender: male
Enter activity level (sedentary, lightly active, moderately active, very active): moderately active
Enter goal weight in kg: 80
New account successfully created
Login to your account
Enter username: ado19
Invalid choice

FUNCTIONS
0. Login/ Sign Up
1. Display User Information
2. Enter food details
3. Display calorie consumption information for the day
4. Display past daily calorie consumption data
5 EXIT

Enter choice: 1

USER INFORMATION
Username: ado19
Name: Adrian
Age: 25
Weight: 90.0
Height: 19